Data Processing for Megatron Bridge LLMs with the DCLM Dataset

Download Data

In [ ]:
from huggingface_hub import login, snapshot_download

# login to HF using token
login(token="input your HF token")

# download dataset
snapshot_download(
    repo_id="mlfoundations/dclm-baseline-1.0",
    repo_type="dataset",
    local_dir="/data/dclm",
    allow_patterns="global-shard_01_of_10/local-shard_0_of_10/**",
    resume_download=True,
    max_workers=32,  # Don't hesitate to increase this number to lower the download time
)

In [ ]:
%%bash

# decompress files
PATH_TO_SAVE=/data/dclm/decompressed
SOURCE_DIR=/data/dclm/global-shard_01_of_10/local-shard_0_of_10
NUM_WORKERS=32

mkdir -p ${PATH_TO_SAVE}
cd ${SOURCE_DIR}
find . -name "*.zst" | parallel -j${NUM_WORKERS} "zstd -d {} -o ${PATH_TO_SAVE}/{.}"

In [ ]:
%%bash

# merge files
SOURCE_DIR=/data/dclm/decompressed
PATH_TO_SAVE=/data/dclm/decompressed/merged.jsonl

cd ${SOURCE_DIR}
awk '1' *.jsonl > ${PATH_TO_SAVE}

# remove small .jsonl files
rm shard_*

In [ ]:
%%bash

# split file before shuffling
LINES_PER_SPLIT=1000000
SOURCE_FILE=/data/dclm/decompressed/merged.jsonl
CHUNKS_DIR=/data/dclm/decompressed/chunks

mkdir -p ${CHUNKS_DIR}
split -l ${LINES_PER_SPLIT} ${SOURCE_FILE} ${CHUNKS_DIR}/chunk_

# shuffle files
NUM_WORKERS=16
SHUFFLE_CHUNKS_DIR=/data/dclm/decompressed/shuffled_chunks
PATH_TO_SAVE=/data/dclm/decompressed/shuffled.jsonl

mkdir -p ${SHUFFLE_CHUNKS_DIR}
ls "${CHUNKS_DIR}"/chunk_* | parallel -j"${NUM_WORKERS}" 'shuf {} -o '"${SHUFFLE_CHUNKS_DIR}"'/$(basename {})_shuf'
rm -rf ${CHUNKS_DIR}
awk '1' ${SHUFFLE_CHUNKS_DIR}/chunk_* > ${PATH_TO_SAVE}
rm -rf ${SHUFFLE_CHUNKS_DIR}
